### 面向对象进阶

在前面的章节我们已经了解了面向对象的入门知识，知道了如何定义类， 如何创建对象以及如何给对象发送消息。

#### @property装饰器
如果想访问属性可以通过属性的getter（访问器）和setter（修改器）方法进行对应的操作。如果要做到这点，就可以考虑使用@property包装器来包装getter和setter方法，使得对属性的访问既安全又方便，代码如下所示。

In [4]:
class Person(object):
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    #getter
    @property
    def name(self):
        return self._name 
    
    @property
    def age(self):
        return self._age 
    
    #setter 只有被set的属性才能修改，否则 AttributeError: can't set attribute
    @age.setter
    def age(self, age):
        self._age = age
        
    def play(self):
        if self._age <= 16:
            print('%s is playing Pachee.' % self._name)
        else:
            print('%s is playing Landlords.' % self._name)
            
            
def main():
    person = Person('小熊女', 12)
    person.play()
    person.age = 22
    person.play()
    # person.name = '大熊' # AttributeError: can't set attribute
    
if __name__ == '__main__':
    main()

小熊女 is playing Pachee.
小熊女 is playing Landlords.


#### __slots__魔法
Python是一门动态语言。
通常，动态语言允许我们在程序运行时给对象绑定新的属性或方法，当然也可以对已经绑定的属性和方法进行解绑定。但是如果我们需要限定自定义类型的对象只能绑定某些属性，可以通过在类中定义__slots__变量来进行限定。需要注意的是__slots__的限定只对当前类的对象生效，对子类并不起任何作用。

In [7]:
class Person(object):
    
    # 限定Person对象只能绑定_name, _age, _gender属性
    __slots__ = ('_name', '_age', '_gender')
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    #getter
    @property
    def name(self):
        return self._name 
    
    @property
    def age(self):
        return self._age 
    
    #setter
    @age.setter
    def age(self, age):
        self._age = age
        
    def play(self):
        if self._age <= 16:
            print('%s is playing Aeroplane Chess.' % self._name)
        else:
            print('%s is playing Landlords.' % self._name)
            
            
def main():
    person = Person('小熊女', 12)
    person.play()
    person._gender = 'W'
    # person._is_gay = True # AttributeError: 'Person' object has no attribute '_is_gay'
    
if __name__ == '__main__':
    main()

小熊女 is playing Aeroplane Chess.


### 静态方法和类方法
写在类中的方法并不需要都是对象方法。

#### 静态方法 @staticmethod

In [10]:
from math import sqrt


class Triangle(object):
    
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
        
    @staticmethod
    def is_valid(a, b, c):
        return a+b>c and a+c>b and b+c>a
    
    def perimeter(self):
        return self._a + self._b + self._c
    
    def area(self):
        half = self.perimeter() / 2
        return sqrt(half * (half - self._a) *  (half - self._b) * (half - self._c))
    

def main():
    a, b, c = 3, 4, 5
    
    if Triangle.is_valid(a, b, c):
        t = Triangle(a, b, c)
        print(t.perimeter())
        print(t.area())
        
         # 也可以通过给类发消息来调用对象方法但是要传入接收消息的对象作为参数
        print(Triangle.perimeter(t))
        print(Triangle.area(t))
    else:
        print('无法构成三角形.')
        
if __name__=='__main__':
    main()

12
6.0
12
6.0


#### 类方法 @classmethod
和静态方法比较类似，Python还可以在类中定义类方法，类方法的第一个参数约定名为cls，它代表的是当前类相关的信息的对象（类本身也是一个对象，有的地方也称之为类的元数据对象），**通过这个参数我们可以获取和类相关的信息并且可以创建出类的对象**，代码如下所示。

In [19]:
from time import time, localtime, sleep


class Clock(object):
    
    def __init__(self, hour=0, minute=0, second=0):
        self._hour = hour
        self._minute = minute
        self._second = second
        
    @classmethod
    def now(cls):
        ctime = localtime(time())
        return cls(ctime.tm_hour, ctime.tm_min, ctime.tm_sec)
    
    def run(self):
        self._second += 1
        if self._second == 60:
            self._second = 0
            self._minute += 1
            if self._minute == 60:
                self._minute = 0
                self._hour += 1
                if self._hour == 24:
                    self._hour =0
                    
    def show(self):
        return '%02d:%02d:%02d' % (self._hour, self._minute, self._second)
    

def main():
    # 通过类方法创建对象并获取系统时间
    clock = Clock.now()
    while True:
        print(clock.show())
        sleep(1)
        clock.run()
        

if __name__ == '__main__':
    main()

13:26:25
13:26:26
13:26:27
13:26:28


KeyboardInterrupt: 

#### 类之间的关系(不是很懂)
简单的说，类和类之间的关系有三种：is-a、has-a和use-a关系。

is-a关系也叫继承或泛化，比如学生和人的关系、手机和电子产品的关系都属于继承关系。
has-a关系通常称之为关联，比如部门和员工的关系，汽车和引擎的关系都属于关联关系；关联关系如果是整体和部分的关联，那么我们称之为聚合关系；如果整体进一步负责了部分的生命周期（整体和部分是不可分割的，同时同在也同时消亡），那么这种就是最强的关联关系，我们称之为合成关系。
use-a关系通常称之为依赖，比如司机有一个驾驶的行为（方法），其中（的参数）使用到了汽车，那么司机和汽车的关系就是依赖关系。

### 继承和多态
刚才我们提到了，可以在已有类的基础上创建新类， 这其中的一种做法就是让一个类从另一个类那里将其属性和方法直接继承下来。从而减少重复代码的编写。
提供继承信息的我们称之为父类，也叫超类或基类；得到继承信息的我们称之为子类，也叫派生类或衍生类。子类除了继承父类的属性和方法，还可以定义自己特有的属性和方法，所以子类比父类拥有更多的能力。在实际开发中，我们经常会用子类对象去替换掉一个父类对象，这是面向对象编程中一个常见的行为，对应的原则称之为**里式替换原则**。

In [4]:
class Person(object):
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    @property
    def name(self):
        return self._name
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        self._age = age
        
    def play(self):
        print('%s is playing.' % self._name)
        
    def watch_hv(self):
        if self._age>=18:
            print('%s is watching Orphan.' % self._name)
        else:
            print('%s is watching SpongeBob SquarePants.' % self._name)
            
            
class Student(Person):
    
    def __init__(self, name, age, grade):
        super().__init__(name, age)
        self._grade = grade
     
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, grade):
        self._grade = grade
        
    def study(self, course):
        print('%s who is a %s student is studying %s.' % (self._name, self._grade, course))
        
        
class Teacher(Person):
    
    def __init__(self, name, age, title):
        super().__init__(name, age)
        self._title = title
        
    @property
    def title(self):
        return self._title
    
    @title.setter
    def title(self, title):
        self._title = title
        
    def teach(self, course):
        print('%s%s is teaching %s.' % (self._name, self._title, course))
        
        
def main():
    stu = Student('plmm', 15, 'Junior')
    stu.study('Math')
    stu.watch_hv()
    
    t = Teacher('宋丹丹', 28, '老师')
    t.teach('演员的自我修养')
    t.watch_hv()
    
    
if __name__ == '__main__':
    main()

plmm who is a Junior student is studying Math.
plmm is watching SpongeBob SquarePants.
宋丹丹老师 is teaching 演员的自我修养.
宋丹丹 is watching Orphan.


#### 父类
子类在继承了父类的方法后，可以对父类已有的方法给出新的实现版本，这个动作称之为方法重写(override)。通过方法重写我们可以让父类的同一个行为在子类中拥有不同的实现版本，当我们调用这个经过子类重写的方法时，不同的子类对象会表现出不同的行为，这个就是多态(poly-morphism)。

#### 抽象类 @abstractmethod

In [5]:
from abc import ABCMeta, abstractmethod


class Pet(object, metaclass=ABCMeta):
    
    def __init__(self, nickname):
        self._nickname = nickname
        
    @abstractmethod
    def make_voice(self):
        pass
    
    
class Dog(Pet):
    
    def make_voice(self):
        print('%s: 汪汪汪汪...' % self._nickname)
        

class Cat(Pet):
    
    def make_voice(self):
        print('%s: 喵喵喵喵...' % self._nickname)
        
        
def main():
    pets = [Dog('Dodge'), Cat('Ketty'), Dog('哮天犬')]
    for p in pets:
        p.make_voice()
        
        
if __name__ == '__main__':
    main()

Dodge: 汪汪汪汪...
Ketty: 喵喵喵喵...
哮天犬: 汪汪汪汪...


上面代码中，Pet类被处理成了一个抽象类。抽象类就是不能实例化的类。这种类的存在就是专门为了让别的类去继承它。Python从语法层面并没有像Java or C# 那样提供对抽象类的支持，但是我们可以通过abc 模块的ABCMeta元类和abstractmethod包装器来达到抽象类的效果。如果一个类中存在抽象方法，name这个类就无法实例化。上面的例子中，Dog and Cat 两个子类分别对Pet类中的make_voice()抽象方法进行了重写并给出了不同的实现版本。当我们在main函数中调用该方法时，这个方法就表现出了多态行为（同样的方法做了不同的事情）。

### 综合案例

#### 复联对抗灭霸

In [13]:
"""
奥特曼打小怪兽

@Author:jyang
@Date:5/16/2019
"""
from abc import ABCMeta, abstractclassmethod
from random import randint, randrange


class Fighter(object, metaclass=ABCMeta):

    __slots__ = ('_name', '_hp')

    def __init__(self, name, hp):
        self._name = name
        self._hp = hp

    @property
    def name(self):
        return self._name

    @property
    def hp(self):
        return self._hp

    @hp.setter
    def hp(self, hp):
        self._hp = hp if hp>=0 else 0

    @property
    def alive(self):
        return self._hp>0

    @abstractclassmethod
    def attack(self, other):
        pass


class Ultraman(Fighter):

    __slots__ = ('_name', '_hp', '_mp')

    def __init__(self, name, hp, mp):
        """

        :param name:
        :param hp: 生命值
        :param mp: 魔法值
        """
        super().__init__(name, hp)
        self._mp = mp

    def attack(self, other):
        other.hp -= randint(15, 25)

    def huge_attack(self, other):
        """
        究极必杀技(打掉对方至少50点或四分之三的血)
        :param other: 被攻击的对象
        :return: 使用成功返回True否则返回False
        """
        if self._mp >= 50:
            self._mp -= 50
            injury = other.hp * 3 // 4
            injury = injury if injury >= 50 else 50
            other.hp -= injury
            return True
        else:
            self.attack(other)
            return False

    def magic_attack(self, others):
        """魔法攻击(群攻)

        :param others: 被攻击的群体
        :return: 使用魔法成功返回True否则返回False
        """
        if self._mp >= 20:
            self._mp -= 20
            for temp in others:
                if temp.alive:
                    temp.hp -= randint(10, 15)
            return True
        else:
            return False

    def resume(self):
        incr_point = randint(1, 10)
        self._mp += incr_point
        return incr_point

    def __str__(self):
        return '~~~%s~~~\n' % self._name + \
            '生命值: %d\n' % self._hp + \
            '魔法值: %d\n' % self._mp


class Avenger(Fighter):

    __slots__ = ('_name', '_hp')

    def attack(self, other):
        other.hp -= randint(10, 20)

    def __str__(self):
        return '~~~%s~~~\n' % self._name + \
               '生命值: %d\n' % self._hp


def is_any_alive(monsters):
    for m in monsters:
        if m.alive:
            return True
    return False

def select_one_alive(monsters):
    monsters_len = len(monsters)
    while True:
        index = randrange(monsters_len)
        monster = monsters[index]
        if monster.alive:
            return monster

def display_info(ultraman, monsters):
    print(ultraman)
    for m in monsters:
        print(m, end='')

def main():
    u = Ultraman('灭霸', 5000, 120)
    m1 = Avenger('Spiderman', 250)
    m2 = Avenger('Ironman', 500)
    m3 = Avenger('StarLord', 750)
    m4 = Avenger('Thors', 750)
    m5 = Avenger('Captain American', 600)
    m6 = Avenger('Hulk', 600)
    ms = [m1, m2, m3, m4, m5]
    fight_round = 1
    while u.alive and is_any_alive(ms):
        print('========Round%02d========' % fight_round)
        m = select_one_alive(ms)
        skill = randint(1, 10)
        if skill <= 6:
            print('%s使用普通攻击打了%s.' % (u.name, m.name))
            u.attack(m)
            print('%s的魔法值恢复了%d点.' % (u.name, u.resume()))
        elif skill <= 9:
            if u.magic_attack(ms):
                print('%s使用了魔法攻击.' % u.name)
            else:
                print('%s使用魔法失败.' % u.name)
        else:
            if u.huge_attack(m):
                print('%s使用究极必杀技虐了%s.' % (u.name, m.name))
            else:
                print('%s使用普通攻击打了%s.' % (u.name, m.name))
                print('%s的魔法值恢复了%d点.' % (u.name, u.resume()))

        if m.alive > 0:
            print('%s回击了%s.' % (m.name, u.name))
            m.attack(u)
        display_info(u, ms)  # 每个回合结束后显示奥特曼和小怪兽的信息
        fight_round += 1

    print('\n========战斗结束!========\n')
    if u.alive > 0:
        print('%s胜利!' % u.name)
    else:
        print('复仇者联盟胜利!')


if __name__ == '__main__':
    main()

========Round01========
灭霸使用普通攻击打了Ironman.
灭霸的魔法值恢复了5点.
Ironman回击了灭霸.
~~~灭霸~~~
生命值: 4982
魔法值: 125

~~~Spiderman~~~
生命值: 250
~~~Ironman~~~
生命值: 476
~~~StarLord~~~
生命值: 750
~~~Thors~~~
生命值: 750
~~~Captain American~~~
生命值: 600
========Round02========
灭霸使用究极必杀技虐了Thors.
Thors回击了灭霸.
~~~灭霸~~~
生命值: 4969
魔法值: 75

~~~Spiderman~~~
生命值: 250
~~~Ironman~~~
生命值: 476
~~~StarLord~~~
生命值: 750
~~~Thors~~~
生命值: 188
~~~Captain American~~~
生命值: 600
========Round03========
灭霸使用普通攻击打了Spiderman.
灭霸的魔法值恢复了10点.
Spiderman回击了灭霸.
~~~灭霸~~~
生命值: 4959
魔法值: 85

~~~Spiderman~~~
生命值: 235
~~~Ironman~~~
生命值: 476
~~~StarLord~~~
生命值: 750
~~~Thors~~~
生命值: 188
~~~Captain American~~~
生命值: 600
========Round04========
灭霸使用普通攻击打了Spiderman.
灭霸的魔法值恢复了6点.
Spiderman回击了灭霸.
~~~灭霸~~~
生命值: 4948
魔法值: 91

~~~Spiderman~~~
生命值: 210
~~~Ironman~~~
生命值: 476
~~~StarLord~~~
生命值: 750
~~~Thors~~~
生命值: 188
~~~Captain American~~~
生命值: 600
========Round05========
灭霸使用普通攻击打了Spiderman.
灭霸的魔法值恢复了3点.
Spiderman回击了灭霸.
~~~灭霸~~~
生命值: 4934
魔法值: 94

~~~Spiderman